In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib as mpl

import math 

from bin import *

from IPython.display import clear_output
%load_ext autoreload
%autoreload 2

In [2]:
mpl.use("pgf")

mpl.rcParams.update({
    "pgf.texsystem": "lualatex",
    'font.family': 'serif',
    'font.size' : 8,
    'text.usetex': True,
    'pgf.rcfonts': False,
    'pgf.preamble': tex_preamble,
})

## Setup

In [3]:
n = 2000
d = 0.3;
σ = 8.0

[a1,b1],[a2,b2] = support_gen(n,d,σ)

λminl = a1-.1
λmaxl = b1+.1
λminr = a2-.1
λmaxr = b2+.1
a = (b1+a2)/2

In [4]:
np.random.seed(0)
A = mat_samp_full(n,d,σ)@np.eye(n)
Λ = np.linalg.eigvalsh(A)
A = np.diag(Λ)

b = np.random.randn(n)
b /= np.linalg.norm(b)

In [5]:
w = a

# norm to measure function approximation and linear system errors
B = np.ones(n)

kappa_w = np.max(np.abs(Λ-w)) / np.min(np.abs(Λ-w))

In [6]:
np.count_nonzero(Λ>a)

1000

In [7]:
K = 111
Q,(a_,b_) = lanczos_reorth(A,b,K,reorth=K)

In [8]:
plot_tol = 1e-14

In [9]:
T = np.zeros((3,K))
T[0,1:] = b_[:-1]
T[1] = a_-w
T[2,:-1] = b_[:-1]

e1 = np.zeros(K+1)
e1[0] = 1

In [10]:
err_CG = np.full(K,np.nan)

for k in range(0,K):
    try:
        e = (1/(Λ-w))*b - np.linalg.norm(b)*Q[:,:k+1]@sp.linalg.solve_banded((1,1),T[:,:k+1],e1[:k+1])
        err_CG[k] = np.sqrt( e.T*B@e )
    except: pass
    
    if err_CG[k] < plot_tol:
        break

In [11]:
res_CG = np.full(K,np.nan)

for k in range(0,K):
    try:
        r = b - np.linalg.norm(b)*(Λ-w)*(Q[:,:k+1]@sp.linalg.solve_banded((1,1),T[:,:k+1],e1[:k+1]))
        res_CG[k] = np.sqrt( r@r )
    except: pass
    
    if res_CG[k] < plot_tol:
        break

## $f(x) = \operatorname{step}(x-a)$, $\Gamma =$ double circles

In [12]:
def f(x):
    return (np.abs(np.real(x))>a)

fAb = f(Λ)*b

In [13]:
err_lanczos = np.full(K,np.nan)
err_quad_form = np.full(K,np.nan)

for k in range(0,K):
    try:
        e = fAb - lanczos_FA(f,Q,a_,b_,k,normb=np.linalg.norm(b))
        err_lanczos[k] = np.sqrt( e.T*B@e )
        err_quad_form[k] = np.abs(b@e)

    except: pass
    
    if err_lanczos[k] < plot_tol:
        break

In [14]:
a_priori_bound = np.full(K,np.inf)
a_posteriori_bound = np.full(K,np.inf)

def Gamma1(t):
    radius = 10*λmaxr-w

    z = w + 1e-10 + 1j*t
    dz = 1j#radius*1j*np.exp(1j*t)
    
    return z,dz

def Gamma2(t):
    radius = 10*(w-λminl)

    z = w-1e-10+1j*t#radius*(np.exp(1j*t)) + (w-radius)
    dz = 1j#radius*1j*np.exp(1j*t)
    
    return z,dz


for k in range(1,K+1):
    a_priori_bound[k-1] = 2*(get_a_priori_bound_Qz(f,Gamma1,[0,np.inf],k,w,λminl,λmaxl,λminr,λmaxr)[0]+\
                             get_a_priori_bound_Qz(f,Gamma2,[0,np.inf],k,w,λminl,λmaxl,λminr,λmaxr)[0])
    a_posteriori_bound[k-1] = 2*(get_a_posteriori_bound_Qz(f,Gamma1,[0,np.inf],a_[:k],b_[:k-1],w,λminl,λmaxl,λminr,λmaxr)[0]+\
                                 get_a_posteriori_bound_Qz(f,Gamma2,[0,np.inf],a_[:k],b_[:k-1],w,λminl,λmaxl,λminr,λmaxr)[0])

In [15]:
fig = plt.figure(figsize=(figure_width*cm,figure_height*cm))
ax = fig.add_axes([left, bottom, width, height])

ks = np.arange(K)

ax.plot(ks,(a_priori_bound*res_CG**2),\
        **line_styles['l3'])
ax.plot(ks,(a_posteriori_bound*res_CG**2),\
        **line_styles['l2'])

ax.plot(ks,(err_lanczos**2),\
        **line_styles['l4'])

ax.plot(ks,err_quad_form,\
        **line_styles['l1'])



ax.set_yscale('log')

ax.grid(True,linestyle=':',linewidth=.5)

#plt.ylim(1e-14,5e-1)

plt.xlabel('number of matvecs: $k$')

plt.savefig('imgs/ch7_step_qf.pdf')
#plt.close()